## Multiplicative Utilities

In [1]:
import numpy as np

In [2]:
def Dmatrices_multiplicative(C,n,p,iprint):
    
    D= np.zeros((p,n,n))
    for i in range(p):
        CC= C[i]
        DD= np.identity(n) - CC
        D[i]= DD
    
    if iprint >= 4:
        print('Print DD matrix ')
        print(D)
    
    return D

In [3]:
def Diteration_multiplicative(D,f,n,p,K,iprint):
    
    vstore=np.zeros((n*p*K,1))
    v=np.zeros((n,1))
    ff=np.zeros((1,n))

    for k in range(K):
        for i in range (p):
            DD= D[i]
            ff= f[i,range(n)]
            ff= ff[np.newaxis]
            v= DD@v+ff.T
            vstore[k*n*p + i*n : k*n*p + (i+1)*n]= v[range(n)] 
    
            if iprint >= 4:
                print('Outer iteration k, Inner iteration i',k,i) 
                print(v.T)
        
        if iprint >= 2:
            print('Outer iteration ',k)
            print(v.T)
        
    return vstore

### ST Iteration Multiplicative

In [4]:
def Smatrix_multiplicative(D,n,p,iprint):
    # Construct S matrix for multiplicative Schwarz

    S= np.zeros((n*p,n*p))
    for i in range(0,p):
        # print((i)*n,(i+1)*n, (i)*n,(i+1)*n)
        S[(i)*n:(i+1)*n, (i)*n:(i+1)*n]= np.identity(n)

    for i in range(1,p):
        DD= D[i]
        # print((i)*n+1,(i+1)*n, (i-1)*n+1,(i)*n)
        S[(i)*n:(i+1)*n, (i-1)*n:(i)*n]= -DD[0:n,0:n]

    if iprint >= 4:
        print('Print S matrix')
        print(S)
    
    return S

In [5]:
def Tmatrix_multiplicative(D,n,p,iprint):
    # Construct T matrix for multiplicative Schwarz

    T= np.zeros((n*p,n*p))
    DD= D[0]
    T[0:n,(p-1)*n:p*n]= -DD

    if iprint >= 4:
        print('Print T matrix')
        print(T)
    
    return T

In [6]:
def gvector_multiplicative(f,n,p,iprint):
    # Construct g vetor for multiplicative Schwarz

    g= np.zeros((n*p,1))
    for i in range(p):
        g[(i)*n:(i+1)*(n),0]= f[i,0:n]

    if iprint >= 4:
        print('Print g vector')
        print(g)
    return g

In [7]:
def STiteration(S,T,g,n,p,K,iprint):
    # Perform multiplicative additive Schwarz iteration using S and T matrices
    # Store solutions from K iterations in a one-dimensional vector

    wstore= np.zeros((n*p*K,1))

    w= np.zeros((n*p))
    w= np.linalg.solve(S,g)
    wstore[0:n*p,0]=w.reshape(n*p)

    for k in range(1,K):
        w= np.linalg.solve(S,(g - T@w))  
        wstore[k*n*p:(k+1)*n*p, 0]= w [0:n*p, 0]
    
        if iprint >= 4:
            print(f'ST iteration # {k:3d}')
  
    return wstore

### U Iteration

In [8]:
def Umatrix_multiplicative(S,T,n,p,K,iprint):
    # Construct U matrix for mutliplicative Schwarz

    U=np.zeros((K*n*p,K*n*p))
    for i in range(K):
        U[i*n*p:(i+1)*n*p, i*n*p:(i+1)*n*p]= S[0:n*p,0:n*p]

    for i in range (K-1):
        U[(i+1)*n*p:(i+2)*n*p, i*n*p:(i+1)*n*p]= T[0:n*p,0:n*p]

    if iprint >= 6:
        print(U)
  
    return U

In [9]:
def hvector_multiplicative(g,n,p,K,iprint):
    # Construct h vector for multiplicative Schwarz

    h= np.zeros((K*n*p,1))
    for i in range(K):
        h[i*n*p:(i+1)*n*p,0]= g[0:n*p,0]

    if iprint >= 4:
        print('h vector')
        print(h)
  
    return h

In [10]:
def Uhsolve_multiplicative(U,h,n,p,K,iprint):
# Perform overlapping multiplicative Schwarz iterations
# K iterations performed by single matrix solve
# Store solutions from K iterations in a one-dimensional vector

    z= np.linalg.solve(U,h)

    if iprint >= 4:
        zz=np.zeros((n*p,K))
        for k in range(K):
            zz[0:n*p,k]=z[(k)*n*p:(k+1)*n*p,0]
 
    return z

### Adjoint Solve

In [11]:
def Diteration_approx(D,f,n,p,K,iprint,mpert,wpert):
  
    # Perform multiplicative Schwarz iteration
    # Compute contribution from each subdomain independently
    # Introduce random error in each subdomain computation
    #
    # Store exact solutions from K iterations in a one-dimensional vector
    # Store approximate solutions from K iterations in a one-dimensional vector
    # Store residuals from K iterations in a one-dimensional vector


    veps=mpert

    DD= np.zeros((n,n))
    ff= np.zeros((n,1))

    v= np.zeros((n,1))
    va= np.zeros((n,1))
    r= np.zeros((n,1))

    # Remember the initial trivial solution
    v_store= np.zeros((n*p*(K+1),1))
    va_store= np.zeros((n*p*(K+1),1))
    r_store= np.zeros((n*p*K,1))
    
#     rand_matrix=np.random.randn(n*p*K,1)

#     if sread:
#         from pandas import read_csv
#         df = read_csv('wpert_multiplicative.csv',header=None)
#         rand_matrix = df.values

#     if swrite:
#         import csv
#         file = open('wpert_multiplicative'+str(iparam)+'.csv', 'w')
#         writer = csv.writer(file)
#         writer.writerows(rand_matrix)
#         file.close()
#     rand_store=np.zeros((n*p*K,1))
#     randout=np.random.randn(n*p*K,1)
#     np.savetxt("rand_out.csv", randout, delimiter=",")#this line is temporary
#     assert(False)
    for k in range(K):
        if iprint >= 2:
            print('Outer iteration ', k)
      
        for i in range (p):
          
            id1= (k+1)*n*p + i*n
            id2= (k+1)*n*p + (i+1)*n

            im1= id1-n
            im2= id2-n
          
            ir1= k*n*p+i*n
            ir2= k*n*p+(i+1)*n
          
            # Compute the contributions from each subdomain
            DD[0:n,0:n]= D[i,0:n,0:n]
            ff[0:n,0]= f[i,0:n]
          
            v[0:n,0]= ff.T + DD @ v[0:n,0]
            v_store[id1:id2,0]= v[0:n,0]

            vpert= veps*wpert[k*p*n + i*n: k*p*n + i*n + n]
            
            va[0:n,0]= ff.T + DD@va[0:n,0] + vpert.T
            va_store[id1:id2,0]= va[0:n,0]
            
            r= ff + DD @ va_store[im1:im2] - va[0:n,0:1]
            r_store[ir1:ir2,0]= r[0:n,0]
          
            if iprint >= 4:
                print(f'Outer iteration {k:3d}, Inner iteration {i:3d}')
                print(va_store)
                

    return v_store,va_store,r_store

In [12]:
def Diteration_adjoint(D,psi,n,p,K,iprint):

    # Perform multiplicative Schwarz iteration
    # Compute contribution from each subdomain independently
    # Introduce random error in each subdomain computation
    #
    # Store exact solutions from K iterations in a one-dimensional vector
    # Store approximate solutions from K iterations in a one-dimensional vector
    # Store residuals from K iterations in a one-dimensional vector

    DD= np.zeros((n,n))
    phi= np.zeros((n,1))
    phi_store= np.zeros((n*p*K,1))

    phi[0:n,0]= psi.reshape(n)
    phi_store[n*p*(K-1)+n*(p-1):n*p*K,0]= psi.reshape(n) #there is proabaly a cleaner way

    for k in range(K,0,-1):
          
        for i in range(p,0,-1):
                                
            id1= n*p*(k-1) + (i-1)*n      
            id2= n*p*(k-1) + i*n
                  
            ip1= id1 + n 
            ip2= id2 + n 
    
                
            # Compute the contributions from each subdomain
            pindex= i
            if i == p:
                pindex= 0
                                
            DD[0:n,0:n]= D[pindex,0:n,0:n]
          
            if iprint >= 2:
                print("k, i, pindex, id1, id2:", k, i, pindex, id1, id2)
          
            if k == K and i == p:
                if iprint >= 2:
                    print('Diteration_adjoint: special case \n')
            else:                
                phi[0:n,0]= DD.T@phi[0:n,0]    
                phi_store[id1:id2]= phi[0:n,0].reshape(n,1)
                if iprint >= 4:
                    print(phi_store)

    return phi_store

In [13]:
def Uhsolve_multiplicative_approx(U,h,n,p,K,iprint,mpert,wpert):
    
    #  Perform multiplicative Schwarz iterations
    #  K iterations performed by single matrix solve
    #  Introduce random error
    #  Store solutions from K iterations in a one-dimensional vector
    # 
    #  Store exact solutions from K iterations in a one-dimensional vector
    #  Store approximate solutions from K iterations in a one-dimensional vector
    #  Store residuals from K iterations in a one-dimensional vector
    

    zexact= np.linalg.solve(U,h)
    zerr= mpert
    
    zapprox= zexact + zerr*wpert
    
    resid= h-U@zapprox
    
    if iprint >= 4:
        zz=np.zeros((n * p, K))
        for k in range(K):
            zz[0:n*p, k]= zexact[k*n*p:(k+1)*n*p,0]
            print(zz[0:n*p,k])

    return zexact,zapprox,resid